# Building a 1D CNN

### Import the relevant packages

In [1]:
from expressyeaself.encode_sequences import encode_sequences_with_method as encode
# import keras

### Define the input data

In [2]:
sample_path = ('/Users/joe.abbott/Documents/dataScience/capstone/ExpressYeaself/example/processed_data/10000_samples\
_from_190523000651814570_deflanked_sequences_inserted_into_Abf1TATA_scaffold_padded_with_exp_levels.txt.gz')


### Encode the sequences

In [12]:
encoded_seqs, exp_levels, max_val = encode(sample_path, scale_els=True, model_type='1DCNN')
print('Shape of encoded sequence array: %s' %(encoded_seqs.shape))
print('Shape of expression level array: %s' %(exp_levels.shape))

TypeError: not all arguments converted during string formatting

### Build the model architecture